# Oracle Table Diagnostics
## WH_LOANS, WH_ACCTCOMMON, WH_ACCT

Identifies:
1. Date columns with values outside .NET range (causes CopyJob DateTime errors)
2. Number/Decimal precision issues
3. Sample problematic rows

In [ ]:
# ============================================================
# CONNECTION CONFIGURATION - UPDATE THESE VALUES
# ============================================================

ORACLE_HOST = "your-oracle-server.database.windows.net"  # Oracle server hostname
ORACLE_PORT = "1521"                                      # Usually 1521
ORACLE_SERVICE = "COCCDM"                                 # Service name or SID
ORACLE_USER = "your_username"                             # Oracle username
ORACLE_PASSWORD = "your_password"                         # Oracle password

# Connection string format (update if using different format)
# TNS format: ORACLE_DSN = "(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={host})(PORT={port}))(CONNECT_DATA=(SERVICE_NAME={service})))"
ORACLE_DSN = f"{ORACLE_HOST}:{ORACLE_PORT}/{ORACLE_SERVICE}"

SCHEMA = "COCCDM"

In [ ]:
# Install oracledb if needed (uncomment)
# !pip install oracledb pandas

In [ ]:
import oracledb
import pandas as pd
from datetime import datetime

# Try thin mode first (no Oracle client needed)
oracledb.init_oracle_client()  # Comment this out if using thin mode

print(f"Connecting to {ORACLE_DSN} as {ORACLE_USER}...")

try:
    conn = oracledb.connect(
        user=ORACLE_USER,
        password=ORACLE_PASSWORD,
        dsn=ORACLE_DSN
    )
    print(f"Connected successfully!")
    print(f"Oracle version: {conn.version}")
except Exception as e:
    print(f"Connection failed: {e}")
    print("\nTry thin mode by commenting out oracledb.init_oracle_client()")

In [ ]:
def run_query(sql, desc=""):
    """Run SQL and return DataFrame"""
    print(f"\n{'='*60}")
    print(f"{desc}")
    print(f"{'='*60}")
    try:
        df = pd.read_sql(sql, conn)
        print(df.to_string())
        return df
    except Exception as e:
        print(f"ERROR: {e}")
        return None

## 1. Row Counts

In [ ]:
run_query("""
SELECT 'WH_LOANS' as table_name, COUNT(*) as row_count FROM COCCDM.WH_LOANS
UNION ALL
SELECT 'WH_ACCTCOMMON', COUNT(*) FROM COCCDM.WH_ACCTCOMMON
UNION ALL
SELECT 'WH_ACCT', COUNT(*) FROM COCCDM.WH_ACCT
""", "TABLE ROW COUNTS")

## 2. Date Range Check (Normal Dates)

In [ ]:
run_query("""
SELECT 
    'WH_LOANS' as tbl,
    MIN(RUNDATE) as min_rundate,
    MAX(RUNDATE) as max_rundate,
    COUNT(DISTINCT RUNDATE) as distinct_dates
FROM COCCDM.WH_LOANS
""", "WH_LOANS - RUNDATE Range")

In [ ]:
run_query("""
SELECT 
    'WH_ACCTCOMMON' as tbl,
    MIN(EFFDATE) as min_effdate,
    MAX(EFFDATE) as max_effdate,
    COUNT(DISTINCT EFFDATE) as distinct_dates
FROM COCCDM.WH_ACCTCOMMON
""", "WH_ACCTCOMMON - EFFDATE Range")

In [ ]:
run_query("""
SELECT 
    'WH_ACCT' as tbl,
    MIN(RUNDATE) as min_rundate,
    MAX(RUNDATE) as max_rundate,
    COUNT(DISTINCT RUNDATE) as distinct_dates
FROM COCCDM.WH_ACCT
""", "WH_ACCT - RUNDATE Range")

## 3. INVALID DATE CHECK - WH_LOANS (The Problem Table)
Dates outside .NET range (< 0001-01-01 or > 9999-12-31) cause CopyJob failures

In [ ]:
# Check each date column in WH_LOANS for invalid dates
date_columns_loans = ['RUNDATE', 'ORIGDATE', 'DATEMAT', 'PAIDOFFDATE', 'DATELASTMAINT', 'ADDDATE', 'STOPDATE']

print("WH_LOANS - INVALID DATE CHECK")
print("="*70)

for col in date_columns_loans:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as date_column,
                COUNT(*) as invalid_count,
                MIN({col}) as min_date,
                MAX({col}) as max_date
            FROM COCCDM.WH_LOANS
            WHERE EXTRACT(YEAR FROM {col}) < 1 
               OR EXTRACT(YEAR FROM {col}) > 9999
               OR {col} < TO_DATE('0001-01-01', 'YYYY-MM-DD')
        """, conn)
        invalid = df['INVALID_COUNT'].iloc[0]
        if invalid > 0:
            print(f"\n*** {col}: {invalid} INVALID ROWS ***")
            print(df.to_string())
        else:
            print(f"{col}: OK (0 invalid)")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

In [ ]:
# Find extreme dates (outside reasonable range 1900-2100)
run_query("""
SELECT 
    'ORIGDATE' as col,
    COUNT(*) as extreme_count,
    MIN(ORIGDATE) as min_val,
    MAX(ORIGDATE) as max_val
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM ORIGDATE) < 1900 OR EXTRACT(YEAR FROM ORIGDATE) > 2100
UNION ALL
SELECT 'DATEMAT', COUNT(*), MIN(DATEMAT), MAX(DATEMAT)
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM DATEMAT) < 1900 OR EXTRACT(YEAR FROM DATEMAT) > 2100
UNION ALL
SELECT 'PAIDOFFDATE', COUNT(*), MIN(PAIDOFFDATE), MAX(PAIDOFFDATE)
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM PAIDOFFDATE) < 1900 OR EXTRACT(YEAR FROM PAIDOFFDATE) > 2100
""", "WH_LOANS - EXTREME DATES (outside 1900-2100)")

In [ ]:
# Sample rows with problematic dates
run_query("""
SELECT ACCTNBR, RUNDATE, ORIGDATE, DATEMAT, PAIDOFFDATE, STATUS, ORIGBAL
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM ORIGDATE) < 1900 
   OR EXTRACT(YEAR FROM ORIGDATE) > 2100
   OR EXTRACT(YEAR FROM DATEMAT) < 1900
   OR EXTRACT(YEAR FROM DATEMAT) > 2100
FETCH FIRST 20 ROWS ONLY
""", "WH_LOANS - SAMPLE ROWS WITH BAD DATES")

## 4. INVALID DATE CHECK - WH_ACCTCOMMON

In [ ]:
date_columns_acctcommon = ['EFFDATE', 'CONTRACTDATE', 'DATEMAT', 'CLOSEDATE', 'DATELASTMAINT']

print("WH_ACCTCOMMON - INVALID DATE CHECK")
print("="*70)

for col in date_columns_acctcommon:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as date_column,
                COUNT(*) as invalid_count,
                MIN({col}) as min_date,
                MAX({col}) as max_date
            FROM COCCDM.WH_ACCTCOMMON
            WHERE EXTRACT(YEAR FROM {col}) < 1 
               OR EXTRACT(YEAR FROM {col}) > 9999
        """, conn)
        invalid = df['INVALID_COUNT'].iloc[0]
        if invalid > 0:
            print(f"\n*** {col}: {invalid} INVALID ROWS ***")
            print(df.to_string())
        else:
            print(f"{col}: OK (0 invalid)")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

## 5. INVALID DATE CHECK - WH_ACCT

In [ ]:
date_columns_acct = ['RUNDATE', 'DATEMAT', 'EFFDATE', 'DATELASTMAINT']

print("WH_ACCT - INVALID DATE CHECK")
print("="*70)

for col in date_columns_acct:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as date_column,
                COUNT(*) as invalid_count,
                MIN({col}) as min_date,
                MAX({col}) as max_date
            FROM COCCDM.WH_ACCT
            WHERE EXTRACT(YEAR FROM {col}) < 1 
               OR EXTRACT(YEAR FROM {col}) > 9999
        """, conn)
        invalid = df['INVALID_COUNT'].iloc[0]
        if invalid > 0:
            print(f"\n*** {col}: {invalid} INVALID ROWS ***")
            print(df.to_string())
        else:
            print(f"{col}: OK (0 invalid)")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

## 6. NUMERIC PRECISION CHECK
Check column definitions for NUMBER precision/scale

In [ ]:
run_query("""
SELECT table_name, column_name, data_type, data_precision, data_scale, nullable
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name = 'WH_LOANS'
  AND data_type = 'NUMBER'
ORDER BY column_id
""", "WH_LOANS - NUMBER COLUMN DEFINITIONS")

In [ ]:
run_query("""
SELECT table_name, column_name, data_type, data_precision, data_scale, nullable
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name = 'WH_ACCTCOMMON'
  AND data_type = 'NUMBER'
ORDER BY column_id
""", "WH_ACCTCOMMON - NUMBER COLUMN DEFINITIONS")

In [ ]:
run_query("""
SELECT table_name, column_name, data_type, data_precision, data_scale, nullable
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name = 'WH_ACCT'
  AND data_type = 'NUMBER'
ORDER BY column_id
""", "WH_ACCT - NUMBER COLUMN DEFINITIONS")

## 7. EXTREME NUMERIC VALUES
Check for values that might overflow Decimal types

In [ ]:
numeric_cols_loans = ['ORIGBAL', 'NOTEBAL', 'BOOKBALANCE', 'AVAILBALAMT', 'COBAL', 'PCTPARTSOLD', 'LCRATE', 'OLDPI', 'PF']

print("WH_LOANS - NUMERIC VALUE RANGES")
print("="*70)

for col in numeric_cols_loans:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as column_name,
                MIN({col}) as min_val,
                MAX({col}) as max_val,
                COUNT(*) as non_null_count
            FROM COCCDM.WH_LOANS
            WHERE {col} IS NOT NULL
        """, conn)
        print(f"{col}: min={df['MIN_VAL'].iloc[0]}, max={df['MAX_VAL'].iloc[0]}, count={df['NON_NULL_COUNT'].iloc[0]}")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

## 8. ALL DATE COLUMNS - FULL SCHEMA CHECK

In [ ]:
run_query("""
SELECT table_name, column_name, data_type
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name IN ('WH_LOANS', 'WH_ACCTCOMMON', 'WH_ACCT')
  AND data_type = 'DATE'
ORDER BY table_name, column_id
""", "ALL DATE COLUMNS IN TARGET TABLES")

## 9. SUMMARY

In [ ]:
print("\n" + "="*70)
print("DIAGNOSTIC SUMMARY")
print("="*70)
print(f"\nRun completed at: {datetime.now()}")
print("\nNext steps:")
print("1. Review any columns with INVALID rows above")
print("2. Check NUMBER columns with NULL precision (unlimited)")
print("3. Use CASE WHEN in CopyJob query to handle bad dates")
print("\nPush this notebook output back to the repo for analysis.")

In [ ]:
# Close connection
conn.close()
print("Connection closed.")